In [ ]:
!pip install opencv-python

In [ ]:
import numpy as np
# import cv2
from matplotlib import pyplot as plt


In [ ]:
image = cv2.imread("melanoma.jpeg",cv2.IMREAD_GRAYSCALE)
plt.imshow(image,cmap='gray')
plt.show()
image.shape

In [18]:
class sequential(): 
  def __init__(self,):
    self.output_shape = ()
    self.Graph = []
  def add(self,Object):
    self.Graph.append(Object)
    print(self.Graph)
    self.output_shape =  Object.output_shape(    num_filters = Object.num_filters ,
                                                 filter_size = Object.filter_size ,
                                                 input_shape = Object.input_shape 
                                                 ) 
   
  def cnn(self,num_filters,filter_size,input_shape = None):
    if(input_shape == None):
      if(len(self.Graph)== 0):
        raise("the operations graph in empty")
      else:
        temp = cnn(input_shape = self.output_shape,num_filters = num_filters,filter_size = filter_size)
    else:
      temp = cnn(input_shape = input_shape,num_filters = num_filters,filter_size = filter_size)
    return temp
  def max_pool(self,num_filters,filter_size,input_shape = None):
    if(input_shape == None):
      if(len(self.Graph)== 0):
        raise ValueError("the operations graph in empty")
      else:
        temp = maxPool(self.output_shape,filter_size)
    else:
      temp = maxPool(input_shape,filter_size)
    return temp
  def forawrdProp(self,image):
   image = image
   for i in self.Graph:
      image = i.forwardProp(image)
   self.out = image
   return self.out 
  def dense(self,output_size,input_shape = None):
    if(input_shape == None):
      if(len(self.Graph)== 0):
        raise("the operations graph in empty")
      else:
        temp = dense(self.output_shape,output_size)
    else:
      temp = dense(input_shape,output_size)
    return temp
  def backProp(self,input_,lable,learn_rate):
#       print("from back prop sequential")
      self.forawrdProp(input_)
      delta = self.Graph[-1].backProp(lable = lable,learn_rate=learn_rate)
      for i in range(2,len(self.Graph)+1):
        delta = self.Graph[-i].backProp(gradient = delta,learn_rate = learn_rate)
     
      return True

  def fit(self,input_,lables,learn_rate,batch_size,epochs):
    for epoch in range(epochs):
        for k,(x,y) in enumerate(zip(input_,lables)):
            self.backProp(x,y,learn_rate)
            if((k+1)%batch_size == 0):
#                 print("after batch ends")
                self.update(learn_rate)
#         print("after epoch ends")
        self.update(learn_rate)
        
  def update(self,learn_rate):
#     print("from update")
    for i in self.Graph:
        
        i.update(learn_rate)

In [19]:
class cnn():
  def __init__(self,input_shape,num_filters,filter_size):
    self.num_filters = num_filters
    self.filter_size = filter_size
    self.input_shape = input_shape
    self.conv_filters = np.random.randn(num_filters,filter_size,filter_size)/(filter_size*filter_size)
  def forwardProp(self,image):
    height, width = image.shape
    self.conv_out= np.zeros((self.num_filters,(height - self.filter_size + 1) ,(width - self.filter_size + 1 )))
    for image_patch,j,i in self.image_region(image):
      for k in range(self.num_filters):
        if (np.sum(image_patch * self.conv_filters[k] ,axis=(0,1))<0) : 
          self.conv_out[k,j,i] = 0
        else: 
          self.conv_out[k,j,i] = np.sum(image_patch * self.conv_filters[k] ,axis=(0,1))
    for i in range(self.num_filters):
      plt.imshow(self.conv_out[i,:,:],cmap='gray')
      plt.show()    

    return self.conv_out
  def image_region(self,image,filter_=None):
    if filter_ is None:
      filter_size = self.filter_size
    else:
      filter_size = filter_.shape[1]
    self.image = image
    hight,width = image.shape
    for i in range(hight- filter_size+1):
      for j in range(width-filter_size+1):
        image_patch = image[i:(i+filter_size),j:(j+filter_size)]
        yield image_patch,i,j
  def output_shape(self,input_shape,filter_size,num_filters):
    if(len(input_shape) == 2):
      return (num_filters,(input_shape[-2]-filter_size + 1 ),(input_shape[-1]-filter_size+1))
    elif(len(input_shape) == 3):
      return (num_filters * input_shape[0],(input_shape[-2]-filter_size+1),(input_shape[-1]-filter_size+1))
    """ def backProp(self,gradient,learn_rate):
    self.dl_dout = np.zeros(self.input_shape)
    self.dl_d_filter = np.zeros(self.conv_filters.shape)
    for image_patch,j,i in self.image_region(image,gradient):
      for k in range(self.num_filters):
        self.dl_d_filter[k,i,j] = np.sum(image_patch*gradient[k] ,axis = (0,1) )  
      
    self.conv_filters = np.array([f-learn_rate*(nf/np.amax(nf,axis=(0,1))) for f,nf in zip(self.conv_filters,self.dl_d_filter)])
   for k in range(self.num_filters):
      self.conv_filters = self.conv_filters/np.amax(self.conv_filters,axis=(0,1))"""

  def backProp(self,gradient,learn_rate):
    dl_d_conv = np.zeros(self.conv_filters.shape)
    """    
    for image_patch,i,j in self.image_region(self.image):
      for k in range(self.num_filters):
        dl_d_conv[k] += image_patch*gradient[k,i,j]
    self.conv_filters -=  learn_rate*dl_d_conv   
   """

    for k in range(gradient.shape[0]):   # On parcourt tous les filtres
      for i in range(self.filter_size): # indices du résultat
        for j in range(self.filter_size):
          for l in range(gradient.shape[1]):
            for c in range(gradient.shape[2]): 
              dl_d_conv[k,i,j] += (gradient[k,l,c] * self.image[ i+l, j+c])
    self.conv_filters = np.array([w-(learn_rate * nw)
                          for w,nw in zip(self.conv_filters,dl_d_conv)]).reshape(self.conv_filters.shape)
#     print(dl_d_conv)
    return self.conv_filters 

In [ ]:
class maxPool():
  def __init__(self,input_shape,filter_size):
    self.filter_size = filter_size
    self.num_filters = input_shape[0]
    self.input_shape = input_shape
  def output_shape(self,input_shape,filter_size,num_filters):
      return (num_filters ,(input_shape[-2]//filter_size ),(input_shape[-1]//filter_size))
  def image_region(self,image):
    self.image = image
    hight = image.shape[1]//self.filter_size
    width = image.shape[2]//self.filter_size
    for k in range(self.num_filters):
      for i in range(hight):
        for j in range(width):
          image_patch = image[k,(i*self.filter_size):(i*self.filter_size+self.filter_size),(j*self.filter_size):(j*self.filter_size+self.filter_size)]
          yield image_patch,k,i,j
  def forwardProp(self,image):
    num_filters,hight,width= image.shape
    self.output = np.zeros((num_filters,hight // self.filter_size, width // self.filter_size  ))
    for image_patch,k,i,j in self.image_region(image):
      self.output[k][i][j] = np.amax(image_patch,axis=(0,1))
    return self.output
  def backProp(self,gradient,learn_rate):
    self.dl_dout = np.zeros(self.image.shape)
    for image_patch,k,i,j in self.image_region(self.image):
      hight,width = image_patch.shape
      max_val = np.amax(image_patch,axis=(0,1))
      for i1 in range(hight):
        for j1 in range(width):
          if(max_val == image_patch[i1,j1]):
            
            self.dl_dout[k,i*self.filter_size+i1,j*self.filter_size+j1] = gradient[k,i,j]
#     print(self.dl_dout.shape[0])

    return self.dl_dout  



In [20]:
class FilterAvgPool():
  def __init__(self,input_shape,filter_size):
    self.filter_size = filter_size
    self.num_filters = input_shape[0]
    self.input_shape = input_shape
  def output_shape(self,input_shape,filter_size,num_filters):
      return (num_filters ,(input_shape[-2]//filter_size ),(input_shape[-1]//filter_size))
  def image_region(self,image):
    self.image = image
    hight = image.shape[1]//self.filter_size
    width = image.shape[2]//self.filter_size
    for k in range(self.num_filters):
      for i in range(hight):
        for j in range(width):
          image_patch = image[k,(i*self.filter_size):(i*self.filter_size+self.filter_size),(j*self.filter_size):(j*self.filter_size+self.filter_size)]
          yield image_patch,k,i,j
  def forwardProp(self,image):
    num_filters,hight,width= image.shape
    self.output = np.zeros((num_filters,hight // self.filter_size, width // self.filter_size  ))
    for image_patch,k,i,j in self.image_region(image):
      self.output[k][i][j] = np.amax(image_patch,axis=(0,1))
    return self.output
  def backProp(self,gradient,learn_rate):
    self.dl_dout = np.zeros(self.image.shape)
    for image_patch,k,i,j in self.image_region(self.image):
      hight,width = image_patch.shape
      max_val = np.amax(image_patch,axis=(0,1))
      for i1 in range(hight):
        for j1 in range(width):
          if(max_val == image_patch[i1,j1]):
            
            self.dl_dout[k,i*self.filter_size+i1,j*self.filter_size+j1] = gradient[k,i,j]
#     print(self.dl_dout.shape[0])

    return self.dl_dout  

In [ ]:
class dense():
  def __init__(self,input_shape,output_size):
    if(len(input_shape) == 1):
      self.input_shape = input_shape[0]
    else: 
      self.input_shape = 1
      for i in input_shape:
        self.input_shape *= i 
      
    self.inp_origin_shape = input_shape
    self.num_filters = 0
    self.filter_size = 0
    self.output_size = output_size
    self.weights = np.random.randn(self.input_shape,output_size)
    self.biases = np.random.randn((self.output_size))
    self.db = np.zeros(self.biases.shape)
    self.dw = np.zeros(self.weights.shape)
  def output_shape(self,input_shape,filter_size,num_filters):
    #ccomputing the ouput shape to sett up the operations graph
    return (self.output_size,)
  def forwardProp(self,input_):
    if (input_.shape != self.inp_origin_shape):
      raise ValueError("input shape doesn't match the sepicified 1 ")
    self.inp= input_.flatten()
    self.al = np.dot(self.inp , self.weights)+self.biases
    self.zl = self.segmoid(self.al)
    self.out = self.zl
    return self.out
  def segmoid(self,x):
    return 1/(1+np.exp(-x))
  def segmoid_prime(self,x):
    return self.segmoid(x)*(1-self.segmoid(x))
  def backProp(self,learn_rate,lable = None, gradient = None):
    # gradient is the returned value of the next layer during back propagation 
    # it the derivation with respect to the activation 

    if gradient is not None:
      self.gradient = gradient
    elif lable is not None:
      self.gradient = self.zl - lable
    else:
      raise ValueError("No gradient No lable")
    # self.db is the error with respect to biases of this layer 
    # self.dw is the error with respect to weights of this layer 
#     self.db = self.gradient * self.segmoid_prime(self.al)
#     self.dw = np.outer(self.inp,self.db)
    delta = self.gradient * self.segmoid_prime(self.al)
    self.db =self.db + delta
    self.dw =self.dw + np.outer(self.inp,delta)
    self.dal_1 = delta@np.array(self.weights).transpose()
    # updating weights an biases 
#     self.weights = np.array([w-(learn_rate * nw)
#                           for w,nw in zip(self.weights,self.dw)]).reshape(self.weights.shape)
#     self.biases = np.array([b-(learn_rate * nb)
#                           for b,nb in zip(self.biases,self.db)]).reshape(self.biases.shape)
#     self.weights = np.array([w-(learn_rate * nw)
#                           for w,nw in zip(self.weights,self.dw)]).reshape(self.weights.shape)
#     self.biases = np.array([b-(learn_rate * nb)
#                           for b,nb in zip(self.biases,self.db)]).reshape(self.biases.shape)
    
    return self.dal_1.reshape(self.inp_origin_shape)
  def update(self,learn_rate):
    self.weights = np.array([w-(learn_rate * nw)
                          for w,nw in zip(self.weights,self.dw)]).reshape(self.weights.shape)
    self.biases = np.array([b-(learn_rate * nb)
                          for b,nb in zip(self.biases,self.db)]).reshape(self.biases.shape)
    self.db = np.zeros(self.biases.shape)
    self.dw = np.zeros(self.weights.shape)

In [ ]:
cnn1 = sequential()
#


In [ ]:
cnn1.add(cnn1.cnn(num_filters=3,filter_size = 3,input_shape = (224,224)))
cnn1.add(cnn1.max_pool(num_filters=8,filter_size=2))
cnn1.add(cnn1.dense(1000))
cnn1.add(cnn1.dense(1))
for i in range(200):
  cnn1.backProp(input_= image,lable= 1,learn_rate=1)

In [21]:
testback = sequential()
testback.add(testback.dense(input_shape=(3,),output_size = 2))
testback.add(testback.dense(output_size = 5))
testback.add(testback.dense(output_size = 3))
data_train=np.array(([[0,0,0],
             [1,0,0],
             [0,1,0],
             [1,1,0],
             [0,0,1],
             [1,0,1],
             [0,1,1],
             [1,1,1]
             ]) ) 
lable_train = np.array(([[1,1,1],
             [0,1,1],
             [1,0,1],
             [0,0,1],
             [1,1,0],
             [0,1,0],
             [1,0,0],
             [0,0,0]
             ]) )
# for i in range(15000):
#   print("epoch =======> {} ".format(i) )
#   for x,y in zip(data_train,lable_train):
#     testback.backProp(x,y,learn_rate=0.01)

testback.fit(data_train,lable_train,0.25,2,1000)

[<__main__.dense object at 0x00000224095EE438>, <__main__.dense object at 0x00000224114C6780>]
[<__main__.dense object at 0x00000224095EE438>, <__main__.dense object at 0x00000224114C6780>, <__main__.dense object at 0x00000224113EDC18>]


In [22]:
print(testback.forawrdProp(np.array([0,1,0])))

[0.95030296 0.14158834 0.9507822 ]
